In [ ]:
import zipfile
import os

# --- CONFIGURE THESE ---
zip_path = "/content/images.zip"          # Path to your zip file
extract_to = "extracted_folder"     # Folder to extract into

# Create folder if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

# Unzip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Successfully unzipped '{zip_path}' → '{extract_to}'")

Successfully unzipped '/content/images.zip' → 'extracted_folder'


# Experimenting

## Test Tesseract

In [ ]:
!pip install pytesseract faiss-cpu tqdm scikit-learn pandas

import os, re, sqlite3, json
from pathlib import Path
from PIL import Image
import pytesseract                    # OCR engine
from tqdm import tqdm                 # progress bar
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import faiss                          # Vector database
import pandas as pd                   # For final ledger display


# SET FOLDER PATH
base_dir = "/content/extracted_folder/images"

image_files = []
for ext in ["*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG"]:
    image_files.extend(Path(base_dir).rglob(ext))

print(f"✅ Found {len(image_files)} image files in {base_dir}")
if len(image_files) == 0:
    raise FileNotFoundError("⚠️ No images found! Please check your folder path.")


# OCR PROCESSING (extract text from each image)
ocr_results = []

for img_path in tqdm(image_files, desc="🧠 Running OCR"):
    img = Image.open(img_path)                             # open image
    text = pytesseract.image_to_string(img)                 # run OCR
    ocr_results.append({"file": img_path.name, "raw_text": text})  # store results

# Print sample OCR output for first image
print("\n🧾 Sample OCR Output:\n", ocr_results[0]["raw_text"][:300])


# PARSE STRUCTURED FIELDS (store name, date, total)
def parse_receipt_text(raw_text):
    total_match = re.search(r"[¥$]\s?([\d,]+\.\d{2})", raw_text)    # find total price
    date_match = re.search(r"(\d{4}[/-]\d{1,2}[/-]\d{1,2})", raw_text)  # find date
    items = re.findall(r"([A-Za-z\s]+)\s[¥$]\s?[\d,]+\.\d{2}", raw_text)  # list items
    return {
        "store": raw_text.split("\n")[0].strip()[:40],                # assume first line = store
        "date": date_match.group(1) if date_match else None,          # date or None
        "total": float(total_match.group(1).replace(",", "")) if total_match else None,  # total or None
        "items": items,
        "raw_text": raw_text
    }

structured_receipts = [parse_receipt_text(r["raw_text"]) for r in ocr_results]

# Print example structured JSON
print("\n📦 Structured data example:\n", json.dumps(structured_receipts[0], indent=2))


# TEXT EMBEDDING (TF-IDF)
texts = [r["raw_text"] for r in structured_receipts]              # get all texts
vectorizer = TfidfVectorizer(max_features=256)                    # limit to 256 features
embeddings = vectorizer.fit_transform(texts).toarray().astype("float32")
print("\n📊 Embedding shape:", embeddings.shape)


# BUILD VECTOR DATABASE (FAISS)
dim = embeddings.shape[1]                                          # vector dimension
index = faiss.index_factory(dim, "IVF10,Flat")                     # create IVF index
index.train(embeddings)                                            # train index
index.add(embeddings)                                              # add vectors

# Example similarity search
D, I = index.search(embeddings[0:1], k=min(3, len(image_files)))   # find top-3 similar receipts
print("\n🔍 Nearest neighbors for first receipt:", I)
print("Distances:", D)


# RULE-BASED ORCHESTRATOR (LLM SIMULATION)
def orchestrate_entry(receipt):
    store = receipt["store"].lower() if receipt["store"] else ""
    if any(k in store for k in ["starbucks", "coffee", "cafe"]):
        category = "Food & Beverages"
    elif any(k in store for k in ["7-eleven", "familymart"]):
        category = "Convenience Store"
    elif any(k in store for k in ["taxi", "uber", "train", "bus"]):
        category = "Transportation"
    else:
        category = "General Expense"
    return {
        "store": receipt["store"],
        "category": category,
        "amount": receipt["total"] or 0.0,
        "date": receipt["date"]
    }

ledger_entries = [orchestrate_entry(r) for r in structured_receipts]

# Show example ledger entry
print("\n💰 Example ledger entry:\n", json.dumps(ledger_entries[0], indent=2))


# SAVE TO SQLITE DATABASE
conn = sqlite3.connect("ledger.db")
cur = conn.cursor()

# Create table if not exists
cur.execute("""
CREATE TABLE IF NOT EXISTS ledger (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    store TEXT,
    category TEXT,
    amount REAL,
    date TEXT
)
""")
conn.commit()

# Insert all entries
cur.executemany("""
INSERT INTO ledger (store, category, amount, date)
VALUES (:store, :category, :amount, :date)
""", ledger_entries)
conn.commit()

# Load ledger into DataFrame
df = pd.read_sql_query("SELECT * FROM ledger", conn)
conn.close()


# DISPLAY FINAL LEDGER
print("\n📘 Final Ledger DataFrame:")
display(df.head(10))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 54.8 MB/s eta 0:00:00
✅ Found 20 image files in /content/extracted_folder/images


🧠 Running OCR: 100%|██████████| 20/20 [00:44<00:00,  2.25s/it]


🧾 Sample OCR Output:
 WHOLE
FOODS

SHARON RD.

PL TORTILLA’S 6.
CAGE FREE ALL WHIT 3
BLACK BEANS 1

Frozen Mangoes 160 2
Whole Strawberries 2
0G LF COTTAGE CHEE 3
8.

2

2.

MAHI MAHI FILLETS
$2 off (1) WC Fill

California Harvest
1.08 1b @ 1.99 /Ib

TARE =
*UT

PLUMS BLACK CV 2.15 B
ITEM = 4040

NHP SLICED OVEN RO 3.99 

📦 Structured data example:
 {
  "store": "WHOLE",
  "date": null,
  "total": null,
  "items": [],
  "raw_text": "WHOLE\nFOODS\n\nSHARON RD.\n\nPL TORTILLA\u2019S 6.\nCAGE FREE ALL WHIT 3\nBLACK BEANS 1\n\nFrozen Mangoes 160 2\nWhole Strawberries 2\n0G LF COTTAGE CHEE 3\n8.\n\n2\n\n2.\n\nMAHI MAHI FILLETS\n$2 off (1) WC Fill\n\nCalifornia Harvest\n1.08 1b @ 1.99 /Ib\n\nTARE =\n*UT\n\nPLUMS BLACK CV 2.15 B\nITEM = 4040\n\nNHP SLICED OVEN RO 3.99 B\nNHP SLICED HICKORY 3.99 B\n156415 Bri eoe7 1b TARE = 01\n\nur GALA APPLES OG 326) B\nITEM = 94135\n\nwee TAX 93 BAL\n\n \n\n45.44\n\f"
}

📊 Embedding shape: (20, 256)

🔍 Nearest neighbors for first receipt: [[ 0 13 -1]]
Dist

,id,store,category,amount,date
0,1,WHOLE,General Expense,0.0,None
1,2,,General Expense,0.0,None
2,3,Yoh,General Expense,0.0,None
3,4,,General Expense,0.0,None
4,5,,General Expense,0.0,None
5,6,See fees bo receipt for your chance,General Expense,0.0,None
6,7,Thornton #629,General Expense,0.0,None
7,8,Thank,General Expense,0.0,None
8,9,See back of receipt for your chance,General Expense,0.0,None
9,10,,General Expense,0.0,None


## Test EasyOCR

In [ ]:
!pip install easyocr faiss-cpu tqdm scikit-learn pandas -q

import os, re, sqlite3, json
from pathlib import Path
import easyocr                         # ← NEW: replaces pytesseract + PIL
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import faiss
import pandas as pd

# INITIALIZE EASYOCR READER
# - lang_list: add languages you expect (e.g., 'en', 'id' for Indonesian)
# - gpu: True = use Colab GPU (much faster)
# - download_enabled: True → auto-download models on first run
reader = easyocr.Reader(['en', 'id'], gpu=True)  # ← change languages as needed


# SET FOLDER PATH & COLLECT IMAGES
base_dir = "/content/extracted_folder/images"

image_files = []
for ext in ["*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG"]:
    image_files.extend(Path(base_dir).rglob(ext))

image_files = [p for p in image_files if p.is_file()]
print(f"Found {len(image_files)} image files in {base_dir}")
if len(image_files) == 0:
    raise FileNotFoundError("No images found! Check your folder path.")


# OCR PROCESSING WITH EASYOCR
ocr_results = []

print("Running EasyOCR on all images...")
for img_path in tqdm(image_files, desc="OCR"):
    # easyocr.readtext returns list of (bbox, text, confidence)
    result = reader.readtext(str(img_path), detail=0, paragraph=True)  # paragraph=True → joins lines
    raw_text = "\n".join(result) if result else ""
    ocr_results.append({"file": img_path.name, "raw_text": raw_text})

# Sample output
print("\nSample OCR Output (first image):\n", ocr_results[0]["raw_text"][:400])


# PARSE STRUCTURED FIELDS (store, date, total, items)
def parse_receipt_text(raw_text):
    lines = [line.strip() for line in raw_text.split("\n") if line.strip()]

    # Store: first non-empty line (common on receipts)
    store = lines[0][:50] if lines else "Unknown"

    # Date: YYYY-MM-DD or DD/MM/YYYY etc.
    date_match = re.search(r"\b(\d{4}[/-]\d{1,2}[/-]\d{1,2}|\d{1,2}[/-]\d{1,2}[/-]\d{4})\b", raw_text)
    date = date_match.group(0) if date_match else None

    # Total: look for keywords + number
    total_match = re.search(r"(?:total|subtotal|amount|pay)[\s:]*([\d,]+\.?\d*)", raw_text, re.I)
    total = None
    if total_match:
        total = float(total_match.group(1).replace(",", ""))

    # Items: "Item Name ... 12.50" → capture name and price
    items = []
    item_pattern = re.compile(r"(.+?)\s{2,}[\d,]+\.?\d*$", re.I)
    for line in lines:
        m = item_pattern.match(line)
        if m:
            name = m.group(1).strip()
            price_match = re.search(r"([\d,]+\.?\d*$)", line)
            if price_match:
                try:
                    price = float(price_match.group(1).replace(",", ""))
                    items.append({"name": name, "price": price})
                except:
                    pass

    return {
        "store": store,
        "date": date,
        "total": total,
        "items": items,
        "raw_text": raw_text
    }

structured_receipts = [parse_receipt_text(r["raw_text"]) for r in ocr_results]

print("\nExample Structured Data:")
print(json.dumps(structured_receipts[0] if structured_receipts else {}, indent=2, default=str))


# TEXT EMBEDDING (TF-IDF)
texts = [r["raw_text"] for r in structured_receipts]
vectorizer = TfidfVectorizer(max_features=256, stop_words='english', ngram_range=(1,2))
embeddings = vectorizer.fit_transform(texts).toarray().astype("float32")
print(f"\nEmbedding shape: {embeddings.shape}")


# BUILD FAISS VECTOR DATABASE (IVF + Flat)
dim = embeddings.shape[1]
nlist = min(10, len(image_files))  # IVF clusters
index = faiss.index_factory(dim, f"IVF{nlist},Flat")
index.train(embeddings)
index.add(embeddings)

# Example: find similar receipts
D, I = index.search(embeddings[:1], k=min(3, len(image_files)))
print("\nNearest neighbors for first receipt:")
for i, (dist, idx) in enumerate(zip(D[0], I[0])):
    print(f"  {i+1}. {image_files[idx].name} (distance: {dist:.4f})")


# RULE-BASED ORCHESTRATOR (category assignment)
def get_category(receipt):
    store = receipt["store"].lower()
    items_text = " ".join([it["name"].lower() for it in receipt["items"]])
    text = store + " " + items_text

    if any(k in text for k in ["starbucks", "coffee", "cafe", "crepe", "momi", "tea", "ham"]):
        return "Food & Beverage"
    elif any(k in text for k in ["7-eleven", "family mart", "indomaret", "alfamart"]):
        return "Convenience Store"
    elif any(k in text for k in ["taxi", "uber", "gojek", "grab", "train", "bus"]):
        return "Transportation"
    elif any(k in text for k in ["hotel", "lodge", "airbnb"]):
        return "Accommodation"
    else:
        return "General Expense"

ledger_entries = []
for r in structured_receipts:
    entry = {
        "file": r.get("file"),  # from OCR loop
        "store": r["store"],
        "category": get_category(r),
        "amount": r["total"] or 0.0,
        "date": r["date"] or "N/A",
        "item_count": len(r["items"])
    }
    ledger_entries.append(entry)

print("\nExample Ledger Entry:")
print(json.dumps(ledger_entries[0], indent=2))


# SAVE TO SQLITE
conn = sqlite3.connect("easyocr_ledger.db")
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS ledger (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    file TEXT,
    store TEXT,
    category TEXT,
    amount REAL,
    date TEXT,
    item_count INTEGER
)
""")

cur.executemany("""
INSERT INTO ledger (file, store, category, amount, date, item_count)
VALUES (:file, :store, :category, :amount, :date, :item_count)
""", ledger_entries)

conn.commit()
df = pd.read_sql_query("SELECT * FROM ledger ORDER BY date DESC, store", conn)
conn.close()


# DISPLAY FINAL LEDGER
print("\nFINAL LEDGER (EasyOCR + FAISS + Rules):")
display(df)

# Export
df.to_csv("easyocr_final_ledger.csv", index=False)
print("\nExported → easyocr_final_ledger.csv")
print("SQLite DB → easyocr_ledger.db")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 8.5 MB/s eta 0:00:00


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteFound 20 image files in /content/extracted_folder/images
Running EasyOCR on all images...


OCR:   0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
OCR:   5%|▌         | 1/20 [00:11<03:36, 11.39s/it]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
OCR:  10%|█         | 2/20 [00:26<04:07, 13.76s/it]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
OCR:  15%|█▌        | 3/20 [00:42<04:07, 14.54s/it]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accel


Sample OCR Output (first image):
 WH;LE FQODS GE Aod
SHARON RD
TORTILLA' $ CAGE FREE ALL VhiT 3 . 69 BLACK BEANS 29 Frozen Mansoes 160 99 Uhole Strawberrles 2 . 99 0g LF COTTAGE CHEE 3 , 49 HAHI HAHI FILLETS 8 . 99 VC 52 off (1 ) UC F1ll 00 - Cal1fornla Harvest 2 . 69 08 1b 99 1b TARE 01 #UT PLUHS BLacK Cv 2 . 15 ITEH 4040
NHP SLIcED Oven RO 3 , 99 NHP SLICED HICKORY 3 , 99 1 , 64 1b 99 7 b TARE 01 W T GaLa APPLES 0G 3 , 26 ITEM 9

Example Structured Data:
{
  "store": "WH;LE FQODS GE Aod",
  "date": null,
  "total": null,
  "items": [],
  "raw_text": "WH;LE FQODS GE Aod\nSHARON RD\nTORTILLA' $ CAGE FREE ALL VhiT 3 . 69 BLACK BEANS 29 Frozen Mansoes 160 99 Uhole Strawberrles 2 . 99 0g LF COTTAGE CHEE 3 , 49 HAHI HAHI FILLETS 8 . 99 VC 52 off (1 ) UC F1ll 00 - Cal1fornla Harvest 2 . 69 08 1b 99 1b TARE 01 #UT PLUHS BLacK Cv 2 . 15 ITEH 4040\nNHP SLIcED Oven RO 3 , 99 NHP SLICED HICKORY 3 , 99 1 , 64 1b 99 7 b TARE 01 W T GaLa APPLES 0G 3 , 26 ITEM 94135\nFrrt Tax\n93\nBAL\n45 . 44"
}



,id,file,store,category,amount,date,item_count
0,9,None,"':!"" ' See back of receipt for your chaince fo...",General Expense,7.00,N/A,0
1,7,None,Cosçg Thornfon #629 16375 N. Washington St Tho...,General Expense,85.61,N/A,0
2,18,None,Give Us feedback 0 survey Thank youi ID #: 7OD...,General Expense,39.68,N/A,0
3,19,None,Give us feedback survey . walmart com Ihark yo...,General Expense,46.44,N/A,0
4,12,None,Givo Ingu M feeslbeck YoUL ID / voy oni3/15/nx...,General Expense,0.00,N/A,0
5,4,None,Iu #,General Expense,50.00,N/A,0
6,8,None,SPAR Te] 036-4481240 enai bergsparuteIkonsa .n...,General Expense,0.00,N/A,0
7,5,None,See back of receipt tor your chance to win $10O0,General Expense,21.74,N/A,0
8,17,None,See back of receipt tor your chance to win $10O0,General Expense,21.74,N/A,0
9,13,None,"See to vin bock ck1800 "" 7e5""ap7nerr Ivigcoué'...",General Expense,0.00,N/A,0



Exported → easyocr_final_ledger.csv
SQLite DB → easyocr_ledger.db


## Test AspriseOcr

In [ ]:
!pip install requests pandas tqdm scikit-learn faiss-cpu openpyxl -q

import requests, json, re, sqlite3, time
from pathlib import Path
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import faiss
import pandas as pd


# CONFIG – ASPRISE OCR (free TEST key)

url = "https://ocr.asprise.com/api/v1/receipt"
api_key = "TEST"                     # free test key
image_folder = Path("/content/extracted_folder/images")

# collect **all** images (png, jpg, jpeg)
image_files = (
    sorted(image_folder.glob("*.png")) +
    sorted(image_folder.glob("*.jpg")) +
    sorted(image_folder.glob("*.jpeg"))
)
image_test = image_files[:3]
image_files = image_test
print(f"Found {len(image_files)} image(s)")


# OCR WITH ASPRISE

def ocr_asprise(img_path: Path) -> dict:
    """Return the first receipt dict from Asprise response."""
    with open(img_path, "rb") as f:
        resp = requests.post(
            url,
            data={'api_key': api_key, 'recognizer': 'auto', 'ref_no': img_path.stem},
            files={'file': f}
        )
    if resp.status_code != 200:
        raise RuntimeError(f"Asprise error {img_path.name}: {resp.text}")
    data = resp.json()
    receipts = data.get("receipts", [])
    return receipts[0] if receipts else {}

print("\nRunning Asprise OCR…")
ocr_raw = []
for p in tqdm(image_files, desc="OCR"):
    raw = ocr_asprise(p)
    ocr_raw.append({
        "file": p.name,
        "raw_text": raw.get("ocr_text", ""),
        "raw_json": raw
    })


# STRUCTURED PARSING (store, date, total, items)

def parse_receipt(text: str) -> dict:
    lines = [l.strip() for l in text.split("\n") if l.strip()]

    # ---- Store ----------------------------------------------------
    store = lines[0][:60] if lines else "Unknown"

    # ---- Date -----------------------------------------------------
    date_match = re.search(r"\b(\d{4}[/-]\d{1,2}[/-]\d{1,2}|\d{1,2}[/-]\d{1,2}[/-]\d{4})\b", text)
    date = date_match.group(0) if date_match else None

    # ---- Total ----------------------------------------------------
    total_match = re.search(r"(?:total|subtotal|amount\s+due)[\s:]*([\d,]+\.?\d*)", text, re.I)
    total = float(total_match.group(1).replace(",", "")) if total_match else None

    # ---- Items (name … price) ------------------------------------
    items = []
    # pattern:  <name>   <spaces>   <price>
    item_pat = re.compile(r"^(.+?)\s{2,}([\d,]+\.?\d*)$", re.I)
    for line in lines:
        m = item_pat.match(line)
        if not m: continue
        name = m.group(1).strip()
        price_str = m.group(2).replace(",", "")
        try:
            price = float(price_str)
        except ValueError:
            continue
        items.append({"name": name, "price": price})

    return {"store": store, "date": date, "total": total, "items": items, "raw_text": text}

structured = [parse_receipt(r["raw_text"]) for r in ocr_raw]
structured = [{**s, "file": r["file"]} for s, r in zip(structured, ocr_raw)]

print("\nSample parsed receipt:")
print(json.dumps(structured[0] if structured else {}, indent=2, default=str))


# TF-IDF EMBEDDINGS

texts = [r["raw_text"] for r in structured]
vectorizer = TfidfVectorizer(max_features=256, stop_words='english', ngram_range=(1,2))
embeddings = vectorizer.fit_transform(texts).toarray().astype('float32')
print(f"Embedding shape: {embeddings.shape}")


# FAISS IVF-Flat INDEX

dim = embeddings.shape[1]
nlist = min(10, len(image_files))                     # number of IVF clusters
index = faiss.index_factory(dim, f"IVF{nlist},Flat")
index.train(embeddings)
index.add(embeddings)

# demo nearest-neighbour for first receipt
D, I = index.search(embeddings[:1], k=min(3, len(image_files)))
print("\nNearest neighbours for first receipt:")
for i, (dist, idx) in enumerate(zip(D[0], I[0])):
    print(f"  {i+1}. {image_files[idx].name} (dist {dist:.4f})")


# RULE-BASED CATEGORY ORCHESTRATOR

def get_category(rec):
    txt = (rec["store"] + " " + " ".join([it["name"] for it in rec["items"]])).lower()
    if any(k in txt for k in ["starbucks","coffee","cafe","crepe","momi","tea","ham","restaurant"]):
        return "Food & Beverage"
    if any(k in txt for k in ["7-eleven","family mart","indomaret","alfamart","mart"]):
        return "Convenience Store"
    if any(k in txt for k in ["taxi","uber","gojek","grab","train","bus","flight"]):
        return "Transportation"
    if any(k in txt for k in ["hotel","lodge","airbnb"]):
        return "Accommodation"
    return "General Expense"

# BUILD PER-ITEM + TOTAL LEDGER

ledger = []
for rec in structured:
    cat = get_category(rec)

    # ---- per-item rows -------------------------------------------------
    for it in rec["items"]:
        name = it["name"]
        price = it["price"]
        qty = 1                                   # Asprise does not give qty → assume 1
        unit_price = price
        ledger.append({
            "file": rec["file"],
            "store": rec["store"],
            "item": name,
            "quantity": qty,
            "unit_price": unit_price,
            "line_total": price,
            "category": cat,
            "date": rec["date"] or "N/A",
            "currency": "IDR" if rec["total"] and rec["total"] > 5000 and "kemang" in rec["raw_text"].lower() else "USD"
        })

    # ---- total row ----------------------------------------------------
    if rec["total"]:
        ledger.append({
            "file": rec["file"],
            "store": rec["store"],
            "item": "**TOTAL**",
            "quantity": None,
            "unit_price": None,
            "line_total": round(rec["total"], 2),
            "category": cat,
            "date": rec["date"] or "N/A",
            "currency": "IDR" if rec["total"] > 5000 and "kemang" in rec["raw_text"].lower() else "USD"
        })

# SAVE TO SQLITE + EXPORT

conn = sqlite3.connect("asprise_ledger.db")
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS ledger (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    file TEXT, store TEXT, item TEXT, quantity INTEGER,
    unit_price REAL, line_total REAL, category TEXT,
    date TEXT, currency TEXT
)
""")
cur.executemany("""
INSERT INTO ledger (file,store,item,quantity,unit_price,line_total,category,date,currency)
VALUES (?,?,?,?,?,?,?,?,?)
""", [
    (e["file"], e["store"], e["item"], e["quantity"],
     e["unit_price"], e["line_total"], e["category"],
     e["date"], e["currency"]) for e in ledger
])
conn.commit()

df = pd.read_sql_query("SELECT * FROM ledger ORDER BY date DESC, id", conn)
conn.close()

print("\nFINAL LEDGER (Asprise OCR + FAISS + Rules)")
display(df)

df.to_csv("asprise_final_ledger.csv", index=False)
df.to_excel("asprise_final_ledger.xlsx", index=False)
print("\nExported → asprise_final_ledger.csv & .xlsx")
print("SQLite DB → asprise_ledger.db")

Found 3 image(s)

Running Asprise OCR…


OCR: 100%|██████████| 3/3 [00:02<00:00,  1.12it/s]


Sample parsed receipt:
{
  "store": "WAL MART\u00ae",
  "date": null,
  "total": 5.11,
  "items": [
    {
      "name": "DISCOUNT GIVEN",
      "price": 0.57
    },
    {
      "name": "SUBTOTAL",
      "price": 5.11
    },
    {
      "name": "TOTAL",
      "price": 5.11
    },
    {
      "name": "CASH TEND",
      "price": 11.0
    },
    {
      "name": "CHANGE DUE",
      "price": 5.89
    }
  ],
  "raw_text": "    WAL MART\u00ae\n       ALWAYS LOW PRICES.\n             Always\n             SUPERCENTER\n            OPEN 24 HOURS\n            MANAGER. TBA\n          ( 515) 986 - 1783\n ST# 5748 OP# 00000158 TE# 14 TR# 03178\n BANANAS     000000004011KF\n   0.41 lb @ 1 lb /0.49         0.20 N\n FRAP         001200010451 F    5.48 N\n             DISCOUNT GIVEN     0.57\n                  SUBTOTAL      5.11\n                     TOTAL      5.11\n                 CASH TEND     11.00\n                 CHANGE DUE     5.89\n      # ITEMS SOLD 2\n      TC# 5679 8348 6485 7828 2003\n     

,id,file,store,item,quantity,unit_price,line_total,category,date,currency
0,1,0.jpg,WAL MART®,DISCOUNT GIVEN,1.0,0.57,0.57,Convenience Store,N/A,USD
1,2,0.jpg,WAL MART®,SUBTOTAL,1.0,5.11,5.11,Convenience Store,N/A,USD
2,3,0.jpg,WAL MART®,TOTAL,1.0,5.11,5.11,Convenience Store,N/A,USD
3,4,0.jpg,WAL MART®,CASH TEND,1.0,11.00,11.00,Convenience Store,N/A,USD
4,5,0.jpg,WAL MART®,CHANGE DUE,1.0,5.89,5.89,Convenience Store,N/A,USD
5,6,0.jpg,WAL MART®,**TOTAL**,NaN,NaN,5.11,Convenience Store,N/A,USD
6,24,10.jpg,SPAR,TOTAL FOR 14 ITEMS,1.0,338.16,338.16,General Expense,N/A,USD
7,25,10.jpg,SPAR,TENDERED Nedbank,1.0,338.16,338.16,General Expense,N/A,USD
8,7,1.jpg,TRADER JOE'S,R-CARROTS SHREDDED 10 OZ,1.0,1.29,1.29,General Expense,06-28-2014,USD
9,8,1.jpg,TRADER JOE'S,R-CUCUMBERS PERSIAN 1 LB,1.0,1.99,1.99,General Expense,06-28-2014,USD



Exported → asprise_final_ledger.csv & .xlsx
SQLite DB → asprise_ledger.db


In [ ]:
#JSON result 1
 {
  "ocr_type": "receipts",
  "request_id": "P_34.150.162.112_mhh3ny4b_2fx",
  "ref_no": "ocr_python_123",
  "file_name": "0.jpg",
  "request_received_on": 1762050990971,
  "success": true,
  "image_width": 612,
  "image_height": 1023,
  "image_rotation": 0.019,
  "recognition_completed_on": 1762050991445,
  "receipts": [
    {
      "merchant_name": "SUPERCENTER OPEN 24 HOURS",
      "merchant_address": "( 515) 986 - 1783",
      "merchant_phone": "+1 515-986-1783",
      "merchant_website": null,
      "merchant_tax_reg_no": null,
      "merchant_company_reg_no": null,
      "region": null,
      "mall": null,
      "country": "US",
      "receipt_no": "03178",
      "date": null,
      "time": null,
      "items": [
        {
          "amount": 0.2,
          "category": null,
          "description": "lb @ 1 lb /0.49",
          "flags": " N",
          "qty": 0.41,
          "remarks": null,
          "tags": null,
          "unitPrice": null
        },
        {
          "amount": 5.48,
          "category": null,
          "description": "FRAP001200010451 F",
          "flags": " N",
          "qty": null,
          "remarks": null,
          "tags": null,
          "unitPrice": null
        },
        {
          "amount": 0.57,
          "category": null,
          "description": "DISCOUNT GIVEN",
          "flags": "",
          "qty": null,
          "remarks": null,
          "tags": null,
          "unitPrice": null
        }
      ],
      "currency": "USD",
      "total": 5.11,
      "subtotal": 5.11,
      "tax": null,
      "service_charge": null,
      "tip": null,
      "payment_method": "cash",
      "payment_details": null,
      "credit_card_type": null,
      "credit_card_number": null,
      "ocr_text": "    WAL MART\u00ae\n       ALWAYS LOW PRICES.\n             Always\n             SUPERCENTER\n            OPEN 24 HOURS\n            MANAGER. TBA\n          ( 515) 986 - 1783\n ST# 5748 OP# 00000158 TE# 14 TR# 03178\n BANANAS     000000004011KF\n   0.41 lb @ 1 lb /0.49         0.20 N\n FRAP         001200010451 F    5.48 N\n             DISCOUNT GIVEN     0.57\n                  SUBTOTAL      5.11\n                     TOTAL      5.11\n                 CASH TEND     11.00\n                 CHANGE DUE     5.89\n      # ITEMS SOLD 2\n      TC# 5679 8348 6485 7828 2003\n      THANK YOU FOR SHOPPING WITH US\n             NATHANIEL BUSH\n          Shop at Walmart & Get\n     Your Back to School List for Less.\n          08/20/10    13:12:01",
      "ocr_confidence": 97.9,
      "width": 457,
      "height": 706,
      "avg_char_width": 11.9889,
      "avg_line_height": 18.658,
      "conf_amount": 83,
      "source_locations": {
        "total": [
          [
            {
              "x": 475,
              "y": 607
            },
            {
              "x": 527,
              "y": 607
            },
            {
              "x": 527,
              "y": 625
            },
            {
              "x": 475,
              "y": 625
            }
          ]
        ],
        "receipt_no": [
          [
            {
              "x": 491,
              "y": 451
            },
            {
              "x": 560,
              "y": 452
            },
            {
              "x": 560,
              "y": 474
            },
            {
              "x": 491,
              "y": 473
            }
          ]
        ],
        "subtotal": [
          [
            {
              "x": 476,
              "y": 584
            },
            {
              "x": 527,
              "y": 585
            },
            {
              "x": 526,
              "y": 605
            },
            {
              "x": 475,
              "y": 604
            }
          ]
        ],
        "merchant_name": [
          [
            {
              "x": 235,
              "y": 386
            },
            {
              "x": 405,
              "y": 385
            },
            {
              "x": 405,
              "y": 406
            },
            {
              "x": 235,
              "y": 407
            }
          ]
        ],
        "doc": [
          [
            {
              "x": 82,
              "y": 148
            },
            {
              "x": 586,
              "y": 158
            },
            {
              "x": 571,
              "y": 935
            },
            {
              "x": 68,
              "y": 926
            }
          ]
        ],
        "merchant_address": [
          [
            {
              "x": 211,
              "y": 430
            },
            {
              "x": 445,
              "y": 431
            },
            {
              "x": 445,
              "y": 451
            },
            {
              "x": 211,
              "y": 450
            }
          ]
        ],
        "merchant_phone": [
          [
            {
              "x": 199,
              "y": 429
            },
            {
              "x": 457,
              "y": 430
            },
            {
              "x": 457,
              "y": 452
            },
            {
              "x": 199,
              "y": 451
            }
          ]
        ]
      }
    }
  ]
}

not yet successful in using LLM orchestrator, but using API for ocr seems better here

# STEP 2 Module Experiment for Extraction

In [ ]:
import re
import json
import pandas as pd
from pathlib import Path
from rapidfuzz import fuzz, process
from sentence_transformers import SentenceTransformer, util

def extract_from_file(file_path):
  # Parameters for detection and parsing
  VENDOR_TOP_LINES = 3               # Number of top lines used to detect vendor name
  FUZZY_THRESHOLD = 70               # Minimum score (0–100) for fuzzy text matching
  SEMANTIC_THRESHOLD = 0.65          # Cosine similarity threshold for semantic detection
  CURRENCY_CHARS = r'[$¥€£₹₩]|Rp'    # Supported currency symbols

  # Regex patterns to extract items (quantity, name, price)
  ITEM_QTY_PATTERNS = [
      r'^(?P<qty>\d+)[xX]?\s+(?P<name>[A-Za-z&\-\s]+?)\s+(?P<price>[¥$€£₹₩Rp]*\s*[\d,]+(?:\.\d{1,2})?)$',
      r'^(?P<name>[A-Za-z&\-\s]+?)\s+(?P<qty>\d+)\s+(?P<price>[¥$€£₹₩Rp]*\s*[\d,]+(?:\.\d{1,2})?)$',
      r'^(?P<name>[A-Za-z&\-\s]+?)\s+(?P<price>[¥$€£₹₩Rp]*\s*[\d,]+(?:\.\d{1,2})?)$'
  ]

  # Keyword groups for total, subtotal, payment, etc.
  KEYWORDS_TOTAL = ["total", "amount due", "grand total", "balance"]
  KEYWORDS_SUBTOTAL = ["subtotal", "sub total"]
  KEYWORDS_CASH = ["cash", "paid", "payment", "received"]
  KEYWORDS_CHANGE = ["change", "return"]

  # LOAD AND CLEAN OCR TEXT
  # -----------------------
  OCR_FILE = Path(file_path)
  if not OCR_FILE.exists():
      raise FileNotFoundError(f"Missing OCR file: {OCR_FILE}")

  raw_text = OCR_FILE.read_text(encoding="utf-8")

  # Remove unwanted symbols and empty lines
  clean_text = re.sub(r'[^\w\s\.\,\-:\/\$\¥\€\£\₹\₩Rp]', ' ', raw_text)
  lines = [ln.strip() for ln in clean_text.splitlines() if ln.strip()]


  # INITIAL RESULT STRUCTURE
  # ------------------------
  result = {
      "vendor": None,
      "date": None,
      "items": [],
      "subtotal": None,
      "total": None,
      "payment_method": None,
      "cash_given": None,
      "change": None,
  }


  # EXTRACT VENDOR NAME
  # -------------------
  # The top few lines usually contain the store or brand name.
  top_lines = " ".join(lines[:VENDOR_TOP_LINES])
  vendor_name = re.sub(r'[\d\|\:\#]', '', top_lines).strip()
  result["vendor"] = vendor_name.title() if vendor_name else "Unknown Vendor"


  # EXTRACT DATE
  # ---------------
  # Common date formats used in receipts
  date_patterns = [
      r'\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b',
      r'\b\d{4}[/-]\d{1,2}[/-]\d{1,2}\b',
      r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s+\d{1,2},?\s+\d{4}\b'
  ]

  for pat in date_patterns:
      if (m := re.search(pat, raw_text, flags=re.IGNORECASE)):
          result["date"] = m.group(0)
          break

  # EXTRACT ITEMS
  # ----------------
  def parse_price(token: str):
      """Convert a currency string (e.g., '$1,200.00') into a float."""
      if not token:
          return None
      cleaned = re.sub(r'[\s'+CURRENCY_CHARS+']', '', token).replace(',', '')
      try:
          return float(cleaned)
      except ValueError:
          return None

  # Parse each line and detect quantity, name, and price
  for line in lines:
      if re.search(r'\b(total|subtotal|tax|cash|change|amount due|balance)\b', line, re.IGNORECASE):
          continue
      matched = False
      for pat in ITEM_QTY_PATTERNS:
          if (m := re.match(pat, line)):
              g = m.groupdict()
              name = g.get("name", "").strip()
              qty = int(g["qty"]) if g.get("qty") and g["qty"].isdigit() else 1
              unit = parse_price(g.get("price"))
              if name and unit is not None:
                  result["items"].append({
                      "name": name,
                      "qty": qty,
                      "unit_price": unit,
                      "line_total": unit * qty
                  })
                  matched = True
                  break
      # Fallback: if not matched, assume last number is price
      if not matched and (f := re.search(r'(.+?)\s+([¥$€£₹₩Rp]?\s*\d[\d,\.]*)$', line)):
          name, price = f.group(1).strip(), parse_price(f.group(2))
          if price is not None:
              result["items"].append({"name": name, "qty": 1, "unit_price": price, "line_total": price})

  # EXTRACT TOTALS AND PAYMENT INFO
  # ----------------------------------
  def fuzzy_find(keywords, threshold=FUZZY_THRESHOLD):
      for ln in lines:
          match = process.extractOne(ln.lower(), keywords, scorer=fuzz.partial_ratio)
          if match and match[1] >= threshold:
              return ln
      return None

  def extract_amount(s):
      if not s:
          return None
      if (m := re.search(r'([¥$€£₹₩Rp]?\s*\d[\d,\.]*)', s)):
          return parse_price(m.group(1))
      return None

  # Match possible lines for each section
  total_line = fuzzy_find(KEYWORDS_TOTAL)
  subtotal_line = fuzzy_find(KEYWORDS_SUBTOTAL)
  cash_line = fuzzy_find(KEYWORDS_CASH)
  change_line = fuzzy_find(KEYWORDS_CHANGE)

  # Extract values directly from matched lines
  res_total = None
  if (m := re.search(r'\b(total|amount due|grand total|balance)\b[^\d]*([¥$€£₹₩Rp]?\s*\d[\d,\.]*)', raw_text, re.IGNORECASE)):
      res_total = parse_price(m.group(2))
  result["total"] = res_total or extract_amount(total_line)

  if (m := re.search(r'\b(subtotal|sub total)\b[^\d]*([¥$€£₹₩Rp]?\s*\d[\d,\.]*)', raw_text, re.IGNORECASE)):
      result["subtotal"] = parse_price(m.group(2))
  elif subtotal_line:
      result["subtotal"] = extract_amount(subtotal_line)

  result["cash_given"] = extract_amount(cash_line) if cash_line else None
  result["change"] = extract_amount(change_line) if change_line else None

  # SEMANTIC FALLBACK (SentenceTransformer)
  # ---------------------------------------
  # If totals were not detected by regex, use semantic similarity
  if result["total"] is None:
      try:
          model = SentenceTransformer('all-MiniLM-L6-v2')
          keywords = ["total amount", "subtotal", "cash paid", "change given"]
          key_vecs = model.encode(keywords, convert_to_tensor=True)
          line_vecs = model.encode(lines, convert_to_tensor=True)
          sim = util.cos_sim(line_vecs, key_vecs)
          for i, line in enumerate(lines):
              if sim[i][0] >= SEMANTIC_THRESHOLD:
                  if (v := extract_amount(line)):
                      result["total"] = v
                      break
      except Exception as e:
          print("Semantic fallback skipped due to error:", e)

  # DETECT PAYMENT METHOD
  # ---------------------
  if re.search(r'\bCASH\b', raw_text, re.IGNORECASE):
      result["payment_method"] = "CASH"
  elif re.search(r'\bCARD\b|\bVISA\b|\bMASTER\b|\bDEBIT\b|\bCREDIT\b', raw_text, re.IGNORECASE):
      result["payment_method"] = "CARD"
  else:
      result["payment_method"] = "UNKNOWN"

  # POST-PROCESSING AND OUTPUT
  # --------------------------
  # Estimate subtotal if missing
  if result["subtotal"] is None and result["items"]:
      result["subtotal"] = sum(i["line_total"] for i in result["items"] if i.get("line_total"))
  if result["total"] is None:
      result["total"] = result["subtotal"]

  # Create DataFrames for display and further use
  items_df = pd.DataFrame(result["items"])
  ledger_df = pd.DataFrame([{
      "vendor": result["vendor"],
      "date": result["date"],
      "subtotal": result["subtotal"],
      "total": result["total"],
      "cash_given": result["cash_given"],
      "change": result["change"],
      "payment_method": result["payment_method"]
  }])

  # Display output
  print("\nStructured JSON Output:")
  print(json.dumps(result, indent=2, ensure_ascii=False))


In [ ]:
extract_from_file("/content/ocr_output.txt")


Structured JSON Output:
{
  "vendor": "Momi  Toy S Creperie Lippo Mall Kemang, L.",
  "date": "25/01/2016",
  "items": [
    {
      "name": "Pai Check :",
      "qty": 1,
      "unit_price": 3850.0,
      "line_total": 3850.0
    },
    {
      "name": "Woman",
      "qty": 1,
      "unit_price": 0.0,
      "line_total": 0.0
    },
    {
      "name": "2 Ham Cheese 14,",
      "qty": 1,
      "unit_price": 0.0,
      "line_total": 0.0
    },
    {
      "name": "1 Ice Java Tea 16,",
      "qty": 1,
      "unit_price": 0.0,
      "line_total": 0.0
    },
    {
      "name": "1 Mineral Water 13,",
      "qty": 1,
      "unit_price": 0.0,
      "line_total": 0.0
    },
    {
      "name": "1 Black   White 72,",
      "qty": 1,
      "unit_price": 0.0,
      "line_total": 0.0
    }
  ],
  "subtotal": 175.0,
  "total": 175.0,
  "payment_method": "CASH",
  "cash_given": 3850.0,
  "change": null
}
